# PROMETEO_v1.0

## Paquetería

In [21]:
import ast
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import shutil
import textwrap
import tiktoken
import umap.umap_ as umap
from dotenv import load_dotenv, find_dotenv
from langchain import PromptTemplate
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from sklearn.mixture import GaussianMixture
from typing import Optional

## Funciones

In [22]:
def num_tokens_from_string(string: str) -> int:
    """Cuenta el número de tokens en el documento
    proporcionado."""
    encoding = tiktoken.get_encoding("cl100k_base")
    num_tokens = len(encoding.encode(string))
    return num_tokens

def reduce_cluster_embeddings(
    embeddings: np.ndarray,
    dim: int,
    n_neighbors: Optional[int] = None,
    metric: str = "cosine",
) -> np.ndarray:
    """Esta función no la comprendo. Estudie
    para entenderla."""
    if n_neighbors is None:
        n_neighbors = int((len(embeddings) - 1) ** 0.5)
    return umap.UMAP(
        n_neighbors=n_neighbors, n_components=dim, metric=metric
    ).fit_transform(embeddings)

def get_optimal_clusters(embeddings: np.ndarray, max_clusters: int = 50, random_state: int = 1234):
    """Obtiene el número óptimo de clústers."""
    max_clusters = min(max_clusters, len(embeddings))
    bics = [GaussianMixture(n_components=n, random_state=random_state).fit(embeddings).bic(embeddings)
            for n in range(1, max_clusters)]
    return np.argmin(bics) + 1

def gmm_clustering(embeddings: np.ndarray, threshold: float, random_state: int = 0):
    """Clusteriza con el método GMM."""
    n_clusters = get_optimal_clusters(embeddings)
    gm = GaussianMixture(n_components=n_clusters, random_state=random_state).fit(embeddings)
    probs = gm.predict_proba(embeddings)
    labels = [np.where(prob > threshold)[0] for prob in probs]
    return labels, n_clusters

def format_cluster_texts(df):
    """Agrupa los textos de cada clúster el listas."""
    clustered_texts = {}
    for cluster in df['Cluster'].unique():
        cluster_texts = df[df['Cluster'] == cluster]['Texto'].tolist()
        clustered_texts[cluster] = " --- ".join(cluster_texts)
    return clustered_texts

def wrap_text_preserve_newlines(text, width=80):
    """Formato para respuestas."""
    lines = text.split('\n')
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]
    wrapped_text = '\n'.join(wrapped_lines)
    return wrapped_text

def process_llm_response(llm_response):
    """Generador de referencias."""
    print(wrap_text_preserve_newlines(llm_response['answer']))
    print('\nReferencias:')
    for contexto in llm_response["context"][:5]:
        print(contexto)

## Parámetros

In [23]:
load_dotenv(find_dotenv())
embeddings = OpenAIEmbeddings(
)

detailed_turbo_llm = turbo_llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo-0125'
)

template = """Tu tarea como revisor bibliográfico profesional
es crear resúmenes extremadamente detallados del siguiente
texto: {text} """

prompt = PromptTemplate.from_template(template)
chain = prompt | detailed_turbo_llm | StrOutputParser()

turbo_llm = ChatOpenAI(
    temperature=0.7,
    model_name='gpt-3.5-turbo-0125'
)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=100
)

## Carga de documentos

In [24]:
# Asegúrate de que haya PDFs en la carpeta 'docs'
documents = DirectoryLoader('./docs/', glob="./*.pdf", loader_cls=PyPDFLoader).load()
# Tratameinto de caracteres indeseados
for d in documents:
    d.page_content = d.page_content.replace('\n', ' ').replace('\t', ' ')

docs = text_splitter.split_documents(documents)
texts = [doc.page_content for doc in docs]

d_sorted = sorted(docs, key=lambda x: x.metadata["source"])
d_reversed = list(reversed(d_sorted))
concatenated_content = "\n\n\n --- \n\n\n".join(
    [doc.page_content for doc in d_reversed]
)
print(
    "Número de tokens en el documento: %s"
    % num_tokens_from_string(concatenated_content)
)

Número de tokens en el documento: 27527


## ¿Quieres crear o cargar una incrustación?

In [25]:
user_input = input("¿Crear (1) o cargar (2) incrustación?: ")
destino_emb = r'c:\Users\luisr\OneDrive\Documentos\GitHub\1day_1thing\prometeo\embed'

if user_input.lower() == "1":
    print(f'Elegiste crear de cero una incrustación.\n')
    global_embeddings = [embeddings.embed_query(txt) for txt in texts]

    embed_name = input('¿Cómo se llama esta incrustación?: ') + '_emb' + '.txt'
    emb = rf'c:\Users\luisr\OneDrive\Documentos\GitHub\1day_1thing\prometeo\{embed_name}'
    with open(rf'./{embed_name}', 'w') as f:
        for i in global_embeddings:
            f.write("%s\n" % i)
    print(f'Estás usando la incrustación: {embed_name}')
    shutil.move(emb, destino_emb)
    
elif user_input.lower() == "2":
    print('Elegiste cargar una incrustación ya creada.\n')
    global_embeddings = []

    embed_name = input('Nombre de la incrustación: ') + '_emb' + '.txt'
    print(f'Estás usando la incrustación: {embed_name}')
    with open(rf'./embed/{embed_name}', 'r') as f:
        for i in f:
            x = ast.literal_eval(i.strip())  # Convertir la cadena a lista de números
            global_embeddings.append(x)

    global_embeddings = np.array(global_embeddings, dtype=float)
    
elif user_input != "1" and user_input != "2":
    print('No seleccionaste ninguna incrustación.\n')



Elegiste crear de cero una incrustación.

Estás usando la incrustación: sociecon_2_emb.txt


## Procesamiento de texto

In [26]:
dim = 2
global_embeddings_reduced = reduce_cluster_embeddings(global_embeddings, dim)
labels, _ = gmm_clustering(global_embeddings_reduced, threshold=0.5)
simple_labels = [label[0] if len(label) > 0 else -1 for label in labels]

df = pd.DataFrame({
    'Texto': texts,
    'Embedding': list(global_embeddings_reduced),
    'Cluster': simple_labels
})

clustered_texts = format_cluster_texts(df)
summaries = {}
for cluster, text in clustered_texts.items():
    summary = chain.invoke({"text": text})
    summaries[cluster] = summary
embedded_summaries = [embeddings.embed_query(summary) for summary in summaries.values()]
embedded_summaries_np = np.array(embedded_summaries)
labels, _ = gmm_clustering(embedded_summaries_np, threshold=0.5)
simple_labels = [label[0] if len(label) > 0 else -1 for label in labels]
clustered_summaries = {}
for i, label in enumerate(simple_labels):
    if label not in clustered_summaries:
        clustered_summaries[label] = []
    clustered_summaries[label].append(list(summaries.values())[i])
final_summaries = {}
for cluster, texts in clustered_summaries.items():
    combined_text = ' '.join(texts)
    summary = chain.invoke({"text": combined_text})
    final_summaries[cluster] = summary
texts_from_df = df['Texto'].tolist()
texts_from_clustered_texts = list(clustered_texts.values())
texts_from_final_summaries = list(final_summaries.values())

combined_texts = texts_from_df + texts_from_clustered_texts + texts_from_final_summaries

## ¿Quieres crear o cargar texto procesado? (hacer)

## 1: Crear

In [27]:
# Nombre del archivo de texto
file_name = input('Nombre del archivo: ') + '.txt'

# Escribir la lista en el archivo de texto
with open(file_name, 'w', encoding='utf-8') as f:
    for t in combined_texts:
        f.write("%s\n" % t)

## 2: Cargar

In [28]:
# Nombre del archivo de texto
file_name = input('Nombre del archivo: ') + '.txt'

# Leer el contenido del archivo y mostrarlo
with open(file_name, 'r', encoding='utf-8') as f:
    content = f.read()

textos = text_splitter.split_text(content)

# Guardar los kb en una carpeta (kbs) (hacer)

### 1. Enseñar

In [29]:
destino_kb = r'c:\Users\luisr\OneDrive\Documentos\GitHub\1day_1thing\prometeo\kbs'
persist_directory = input('¿Cómo se llamará este knowledge-base?: ') + '_kb'
vectorstore = Chroma.from_texts(texts=textos,
                                embedding=embeddings,
                                persist_directory=persist_directory)
vectorstore.persist()
vectorstore = None
os.system(f'zip -r db.zip ./{persist_directory}')

1

### 2. Recordar (si y solo si Enseñar=Done) y pregunta de control

In [30]:
persist_directory = input('¿Qué knoledge-base deseas usar?: ') + '_kb'
embedding = OpenAIEmbeddings()
vectorstore = Chroma(persist_directory=persist_directory, 
                  embedding_function=embedding)

def adjust_final_number(string: str, max_threshold: int, initial_number: int) -> int:
    final_number = initial_number
    while final_number < max_threshold:
        retriever = vectorstore.as_retriever(search_kwargs={"k": final_number})
        docs = retriever.invoke(string)
        text = "".join([doc.page_content for doc in docs])
        if num_tokens_from_string(text) < max_threshold:
            final_number += 1
        else:
            break
    return final_number

final_number = adjust_final_number("¿Cuál es el tema principal del documento?", 10000, 4)
print(f'K final es: {final_number}\n')
retriever = vectorstore.as_retriever(search_kwargs={"k": final_number})

K final es: 26



In [31]:
# Se personaliza el LLM como Prometeo
template = """
Eres Prometeo, un asistente personal de revisión biliográfica que habla Español.

Tu tarea consiste en:

1. Leer detalladamente la información proporcionada en documentos que generalmente son
artículos científicos en formato PDF.

2. Proporcionar respuestas extremadamente detalladas a cualquier tipo de pregunta relacionada 
con el contexto, especialmente a preguntas que involucren peticiones. Evita concluir tus respuestas con un 
párrafo que comience con 'En resumen,...'. Varía las conclusiones de tus respuestas para que 
sean más diversas y creativas.

3. Ser carismático y ofrecer información sobre ti y tus funciones.

SIEMPRE debes responder en Español.

SIEMPRE debes responder con base al siguiente contexto: {context}.

NUNCA hables específicamente del contexto.

NUNCA terminar las respuestas con un resumen o una conclusión predecible.

Teniendo en cuenta TODO lo anterior, responde la siguiente pregunta: {question}
"""

prometeo_prompt = PromptTemplate(
    template=template, input_variables=["context", "question"]
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prometeo_prompt
    | turbo_llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain
)

In [32]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = rag_chain_with_source.invoke(query)
process_llm_response(llm_response)

¿Cuál es la idea de investigación?
La idea de investigación se centra en entender, operacionalizar y medir las
dimensiones distintivas de las Capacidades Dinámicas (DC) para la Innovación en
Modelos de Negocio (BMI). El objetivo es ayudar a la gestión a evaluar las DC de
su empresa para BMI, siguiendo la lógica de la capacidad de respuesta en la
gestión. Es necesario comprender y medir mejor el problema para poder
resolverlo. Por lo tanto, entender la conexión entre las DC, el Modelo de
Negocio (BM) y la Innovación es clave para encontrar formas de gestionar la
creación de estas capacidades.

Referencias:
page_content='El artículo titulado "Abriendo la caja negra de las capacidades dinámicas: Un enfoque para la gestión de la innovación en modelos de negocio en la era digital" fue recibido el 28 de abril de 2021, aceptado el 2 de mayo de 2021 y publicado el 5 de mayo de 2021, con la versión actualizada al 14 de mayo de 2021. Los autores del artículo son Matheus Franco, Vinicius Minatoga

In [33]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = rag_chain_with_source.invoke(query)
process_llm_response(llm_response)

¿Cuál es el objetivo u objetivos del estudio?
El objetivo del estudio es comprender, operacionalizar y medir las dimensiones
distintivas de las Capacidades Dinámicas (DC) para la Innovación en Modelos de
Negocio (BMI). El enfoque está en ayudar a la gestión a evaluar las DC de su
empresa para BMI, siguiendo la lógica de la capacidad de respuesta en la
gestión. Es necesario comprender y medir mejor el problema para poder
resolverlo. Por lo tanto, entender la conexión entre DC, BM y BMI es clave para
encontrar formas de gestionar la creación de DC para BMI.

Referencias:
page_content='Las contribuciones teóricas del estudio incluyen la operacionalización de las DC para BMI y proporcionar una herramienta para medir las DC en la práctica. El estudio destaca la importancia de comprender la conexión entre las DC y BMI y ofrece una hoja de ruta para desarrollar DC en las organizaciones. El estudio también contribuye a la literatura al avanzar en el constructo teórico de DC y su relación con B

In [34]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = rag_chain_with_source.invoke(query)
process_llm_response(llm_response)

¿Cuál es la metodología usada en el estudio?
En el estudio se utilizó una metodología de investigación de diseño,
específicamente el enfoque de Investigación de Ciencia de Diseño (DSR). Este
enfoque consta de cuatro pasos metodológicos: definir la clase de problemas,
realizar una revisión sistemática de la literatura para construir un "artefacto"
que resuelva el problema definido, evaluar el artefacto en la práctica a través
de un estudio de caso y dibujar conclusiones, ajustar el "artefacto" y derivar
implicaciones teóricas y prácticas. Este enfoque permitió diseñar una
herramienta para la evaluación de las Capacidades Dinámicas (DC) para la
Innovación en Modelos de Negocio (BMI), basándose en la revisión sistemática de
la literatura y aplicando la metodología de estudio de caso para validar su
efectividad.

Referencias:
page_content='Las contribuciones teóricas del estudio incluyen la operacionalización de las DC para BMI y proporcionar una herramienta para medir las DC en la práctic

In [35]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = rag_chain_with_source.invoke(query)
process_llm_response(llm_response)

¿Cuáles fueron los resultados más importantes del estudio?
Los resultados más importantes del estudio incluyen la identificación de la
centralidad del modelo de negocio actual en la configuración de las capacidades
dinámicas para la innovación en modelos de negocio. Se destaca que la
herramienta de evaluación, basada en indicadores de capacidades de percepción,
aprovechamiento y transformación, muestra que la acumulación de capacidades
dinámicas depende de actividades y prácticas específicas. Además, se resalta que
la remodelación del modelo de negocio actual también ayuda a promover las
capacidades dinámicas en la empresa. El estudio propone la creación de un 'motor
de innovación en modelos de negocio' para remodelar el modelo de negocio e
incorporar capacidades dinámicas para la creación de valor. En resumen, los
resultados subrayan la importancia de comprender y gestionar de manera efectiva
las capacidades dinámicas para una innovación exitosa en modelos de negocio en
la era digital

In [36]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = rag_chain_with_source.invoke(query)
process_llm_response(llm_response)

¿Puedes explicarme el motor de innovación en modelos de negocio?
El motor de innovación en modelos de negocio es una propuesta presentada en el
artículo para remodelar el modelo de negocio existente e incorporar capacidades
dinámicas para la creación de valor. Se plantea como una función que tiene como
objetivo principal transformar el modelo de negocio actual para adaptarse a las
demandas del entorno digital y promover la innovación. Este motor se basa en
actividades y prácticas específicas que impulsan la evolución de las capacidades
dinámicas de la empresa, permitiendo así la creación de un modelo de negocio más
flexible, adaptativo y orientado hacia la generación de valor a través de la
innovación. En resumen, el motor de innovación en modelos de negocio es una
herramienta estratégica para guiar el proceso de transformación y adaptación del
modelo de negocio hacia un enfoque más innovador y competitivo en la era
digital.

Referencias:
page_content='El resumen del artículo discute c

In [37]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = rag_chain_with_source.invoke(query)
process_llm_response(llm_response)

¿Los resultados cumplen los objetivos de investigación?
Los resultados obtenidos en el estudio cumplen con los objetivos de
investigación establecidos. Se logró una profunda comprensión de las dimensiones
de las capacidades dinámicas y cómo medirlas para la gestión de la innovación en
modelos de negocio. A través de la revisión sistemática de la literatura y el
estudio de caso en la empresa Alpha, se pudo construir una herramienta para
evaluar las capacidades dinámicas. Esta herramienta se evaluó en base a la
práctica, lo que permitió validar su aplicabilidad y efectividad en un entorno
real. Además, se destacó la importancia de adaptar estrategias específicas para
cada empresa y contexto en el desarrollo de capacidades dinámicas para la
innovación en modelos de negocio. En resumen, los resultados alcanzados se
alinean con los objetivos de investigación establecidos, brindando una sólida
base para futuros estudios en el campo de la gestión de la innovación y los
modelos de negocio.

Re

In [ ]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = rag_chain_with_source.invoke(query)
process_llm_response(llm_response)

In [ ]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = rag_chain_with_source.invoke(query)
process_llm_response(llm_response)

In [ ]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = rag_chain_with_source.invoke(query)
process_llm_response(llm_response)

In [ ]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = rag_chain_with_source.invoke(query)
process_llm_response(llm_response)

In [ ]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = rag_chain_with_source.invoke(query)
process_llm_response(llm_response)

# Hacer historial y guardar conversaciones para medir efectividad.